<H1> Utiliser wincopper pour extraire des motifs séquentiels</H1>

L'objectif de ce notebook est de montrer l'utilisation de wincopper. Les algorithmes et structures sont décrites dans 
*Alatrista-Salas, H., Guevara-Cogorno, A., Maehara, Y. Nunez-del-Prado, M. (2020). Efficiently Mining Gapped and Window Constraint Frequent Sequential Patterns. Proceedings 17th International Conference on Modeling Decisions for Artificial Intelligence (https://link.springer.com/chapter/10.1007/978-3-030-57524-3_20)*.   

Wincopper offre la possibilité outre d'extraire les motifs, de pouvoir prendre en compte des contraintes de temps. Il propose également une implémentation de prefixspan. Le lien github est ici : https://github.com/bitmapup/prefixspanr/  

**Attention:** la version actuelle est en Python 2. Une version en Python 3 devrait bientôt être mise en place.

## Installation



Avant de commencer, il est nécessaire de déjà posséder dans son environnement toutes les librairies utiles. Dans la seconde cellule nous importons toutes les librairies qui seront utiles à ce notebook. Il se peut que, lorsque vous lanciez l'éxecution de cette cellule, une soit absente. Dans ce cas il est nécessaire de l'installer. Pour cela dans la cellule suivante utiliser la commande :  

*! pip install nom_librairie*  

**Attention :** il est fortement conseillé lorsque l'une des librairies doit être installer de relancer le kernel de votre notebook.

**Remarque :** même si toutes les librairies sont importées dès le début, les librairies utiles pour des fonctions présentées au cours de ce notebook sont ré-importées de manière à indiquer d'où elles viennent et ainsi faciliter la réutilisation de la fonction dans un autre projet.


**Attention : ici il faut impérativement installer les librairies associées à wincopper**


In [ ]:
# utiliser cette cellule pour installer les librairies manquantes
# pour cela il suffit de taper dans cette cellule : !pip install nom_librairie_manquante
# d'exécuter la cellule et de relancer la cellule suivante pour voir si tout se passe bien
# recommencer tant que toutes les librairies ne sont pas installées ...


!pip install git+https://github.com/bitmapup/prefixspanr.git


# eventuellement ne pas oublier de relancer le kernel du notebook

In [ ]:
# Importation des différentes librairies utiles pour le notebook

#Sickit learn met régulièrement à jour des versions et 
#indique des futurs warnings. 
#ces deux lignes permettent de ne pas les afficher.
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# librairies générales

import pandas as pd
import numpy as np
import re
import sys
import psutil

from ast import literal_eval

# pour wincopper
import wincopper as wc



Pour pouvoir sauvegarder sur votre répertoire Google Drive, il est nécessaire de fournir une autorisation. Pour cela il suffit d'éxecuter la ligne suivante et de saisir le code donné par Google.

In [ ]:
# pour monter son drive Google Drive local
from google.colab import drive
drive.mount('/content/gdrive')

Corriger éventuellement la ligne ci-dessous pour mettre le chemin vers un répertoire spécifique dans votre répertoire Google Drive : 

In [ ]:
my_local_drive='/content/gdrive/My Drive/Colab Notebooks/ML_FDS'
# Ajout du path pour les librairies, fonctions et données
sys.path.append(my_local_drive)
# Se positionner sur le répertoire associé
%cd $my_local_drive

%pwd

# Utilisation de wincopper  


Télécharger le fichier dataset_wincopper.csv dans votre répertoire courant. Pour cela, décommenter la cellule suivante : 

In [ ]:
#!wget http://www.lirmm.fr/~poncelet/Ressources/dataset_wincopper.csv

Lecture et conversion du fichier :

In [ ]:
# les items dans le fichier exemple sont sous la forme de string, i.e. 'a', il faut les convertir en literal
data = pd.read_csv("dataset_wincopper.csv", sep=",", header=0, converters={"sequence": literal_eval})
display(data)


Récupération des identifiants de séquences (*sid*) et des séquences dans des listes.

In [ ]:
sids = list(data["sid"])
sequences = list(data["sequence"])

## Utilisation de PrefixSpan  

Prefixspan manipule des listes d'itemsets. Il est possible de la spécifier dans les options.  

Pour utiliser PrefixSpan, il existe différentes options : 
* *'threshold'* (support minimal) avec *'threshold' : int or float*. Si threshold est un entier Wincopper considère qu'il s'agit d'un support absolu alors que pour un float il s'agit du support relatif. 
* *'items_separated'* avec *'items_separated'= False or True*. Si *itemsSeparated = True* les patterns ne peuvent contenir seulement que des 1-itemsets. Si *itemsSeparated = False* les patterns seront composés de k-itemsets (valeur par défaut).
* *'window'*: taille de fenêtre maximale entre itemsets
* *'gap'*: gap maximum entre itemsets



**Attention:** Prefixspan considère des listes d'itemsets donc *itemsSeparated = False*


In [ ]:
# spécification du support minimal en entier (donc par rapport aux nombres de séquences de la base)
threshold = 3
# contient k-itemsets séparés par des virgules
items_separated = False

options = {'threshold': threshold, 'itemsSeparated': items_separated}

Il suffit alors d'appeler la méthode *prefixspan* avec la liste des options.

In [ ]:
result_mining = wc.prefixspan(sequences, options)

for pat in result_mining:
    print(pat)

Avec d'autres contraintes : 


In [ ]:
# spécification du support minimal en entier (donc par rapport aux nombres de séquences de la base)
threshold = 3
# contient k-itemsets séparés par des virgules
items_separated = False
# max window de 1, il faut 1 au max entre les itemsets
maxwindow=1
options = {'threshold': threshold, 'itemsSeparated': items_separated, 'window':maxwindow}
result_mining = wc.prefixspan(sequences, options)

for pat in result_mining:
    print(pat)

## Utilisation de wincopper 

Wincopper propose de pouvoir utiliser différentes contraintes qui peuvent être précisées dans les options : 

* *'threshold'*: support minimal en entier (support absolu) ou en float (support relatif)
* *'minSseq'*: constrainte de taille minimale minimale des itemsets (itemset size)
* *'maxSseq'*: contrainte de taille maximale des itemsets  (itemset size)
* *'minSize'*: contrainte de taille minimale des patterns (subsequence size)
* *'maxSize'*: contrainte de taille maximale des patterns (subsequence size)
* *'logic'*: contraite d'inclusion souple   
OR relation '(s1 | s2)'  
AND relation '(s1 & s2)'  

In [ ]:
# spécification du support minimal en entier (donc par rapport aux nombres de séquences de la base)
threshold = 2
# contient k-itemsets séparés par des virgules
items_separated = False

# taille minimale des sous séquences
minseq = 2
# taille maximale des sous séquences
maxseq = 2
options = {'threshold': threshold, 'itemsSeparated': items_separated, 'maxSseq':maxseq,'minSseq':minseq}

result_mining = wc.prefixspan(sequences, options)

for pat in result_mining:
    print(pat)

Il existe d'autres options qui peuvent être utilisées :
* *'dataDesc'*: permet de donner un nom au fichier résultat
* *'resultFile'*: drapeau pour générer un fichier résultat. Si *resultFile=True* un fichier avec les patterns extrait est créé (défaut). Si *resultFile=False* aucun fichier n'est généré.
* *'test'*: pour générer un résumé de fichier de tests (dans un but d'expérimentation). Si *test=True* le résumé est généré. Si *test=False* aucun fichier n'est généré (défaut).


In [ ]:
# spécification du support minimal en entier (donc par rapport aux nombres de séquences de la base)
threshold = 2
# contient k-itemsets séparés par des virgules
items_separated = False

# taille minimale des sous séquences
minseq = 2
# taille maximale des sous séquences
maxseq = 2
# 
test=True
options = {'threshold': threshold, 'itemsSeparated': items_separated, 'maxSseq':maxseq,'minSseq':minseq, 
          'test':True, 'dataDesc':'toto.txt'}

result_mining = wc.prefixspan(sequences, options)

for pat in result_mining:
    print(pat)